### Ingest drivers.json file

### Step 1 - Read the JSON file using spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                StructField("surname", StringType(), True)])

In [0]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                             StructField("driverRef", StringType(), True),
                             StructField("number", IntegerType(), True),
                             StructField("code", StringType(), True),
                             StructField("name", name_schema),
                             StructField("dob", DateType(), True),
                             StructField("nationality", StringType(), True),
                             StructField("url", StringType(), True)])

In [0]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json("abfss://raw@formula1dl1216.dfs.core.windows.net/drivers.json")

In [0]:
drivers_df.printSchema()

In [0]:
display(drivers_df)

### Step 2 - Rename columns and add new columns 
1. driverid renamed to driver_id
2. driverRef renamed to driver_ref
3. ingestion date added
4. name added with concatenation of forename and surname

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("driverRef", "driver_ref") \
.withColumn("ingestion_date", current_timestamp()) \
.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

### Step 3 - Drop the unwanted columns
1. name.forename
2. name.surname
3. url

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

### Step 4 - Write to output to processed container in parquet format

In [0]:
drivers_final_df.write.mode("overwrite").parquet("abfss://raw@formula1dl1216.dfs.core.windows.net/constructors.json")

In [0]:
display(spark.read.parquet("abfss://raw@formula1dl1216.dfs.core.windows.net/constructors.json"))